<a href="https://colab.research.google.com/github/anaisidoro/ASOCIO-2025---POwer-Girls/blob/main/Instance6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
!pip install pulp
from pulp import *
import time
# --- INÍCIO DA MEDIÇÃO ---
start_time = time.time()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 98.7 MB/s eta 0:00:00


In [2]:
#Input data
with open('/content/instance6.json') as f:
    instance = json.load(f)

#Sets and parameters
days = ["L", "Ma", "Mi", "J", "V"]
employee = instance['Employees']
desks = instance['Desks']
zones = instance['Zones']
desks_per_zone = instance['Desks_Z']
desks_per_employee = instance['Desks_E']
employee_per_group = instance['Employees_G']
preferable_days = instance['Days_E']
weight = 5
groups = [g for g in employee_per_group]

#Update weight matrix
weights = [[1 for _ in employee] for _ in days]
for e in employee:
    for i in range(len(preferable_days[e])):
        day_index = days.index(preferable_days[e][i])
        weights[day_index][employee.index(e)]=weight

#Identify the employees that can use each desk
employee_per_desk = {}
for desk_id in desks:
    employees_in_desk = []
    for e in employee:
        possible_desks = desks_per_employee[e]
        for m in possible_desks:
            if m == desk_id:
                employees_in_desk.append(e)
    employee_per_desk[desk_id]=employees_in_desk

In [3]:
#Problem Initialization
assignment = LpProblem("StudentChallenge", LpMaximize)

#Decision Variables
x = LpVariable.dicts("x", [(d,e,k) for d in days for e in employee for k in desks if e in employee_per_desk[k]], cat = "Binary")
u = LpVariable.dicts("u", [(d,g) for d in days for g in groups], cat = "Binary")

#Objective Function
assignment += lpSum([weights[days.index(d)][employee.index(e)]*x[d,e, k] for d in days for e in employee for k in desks if e in employee_per_desk[k]])

#Constraint 1 = Each employee must stay at the office at least twice a week
for e in employee:
    assignment += lpSum([x[d,e,k] for d in days for k in desks if e in employee_per_desk[k]]) >= 2

#Constraint 2 = Each desk must be occupied by at most one specific employees at each day
for k in desks:
    for d in days:
        assignment += lpSum([x[d,e,k] for e in employee_per_desk[k]]) <= 1

#Constraint 3 - Each employee must be assigned at most once a day
for d in days:
    for e in employee:
        assignment += lpSum([x[d,e,k] for k in desks if e in employee_per_desk[k]]) <= 1

#Constraint 4 - Each desk must be assigned at most once a day
for k in desks:
    for d in days:
        assignment += lpSum([x[d,e,k] for e in employee if e in employee_per_desk[k]]) <= 1


#Constraint 5 = Employees should not be isolated in a zone
for d in days:
    for z in zones:
        for g in groups:
            for m in employee_per_group[g]:
                assignment += lpSum([x[d,m,k] for k in desks_per_employee[m] if k in desks_per_zone[z]]) <= lpSum([x[d,e,k] for e in employee_per_group[g] if e!=m for k in desks_per_employee[e] if k in desks_per_zone[z]])


#Constraint 6 = Each group should have one meeting per week
for g in groups:
    assignment += lpSum([u[d,g] for d in days]) >= 1

#Constraint 7 = All the members of a group should be present once a week
for d in days:
    for g in groups:
        employees_group = employee_per_group[g]
        assignment += lpSum([x[d,e,k] for e in employees_group for k in desks if e in employee_per_desk[k]]) >= len(employees_group)*u[d,g]

In [4]:
#Solving the model
solve_the_model = assignment.solve(PULP_CBC_CMD(timeLimit=3600, keepFiles=True))
print(f"Status do problema: {LpStatus[solve_the_model]}")

#Print variables
for var in assignment.variables():
    if var.varValue > 0:
        print(f"{var.name}={var.varValue}")

#Print objective function value
print(f"Total Happiness: {value(assignment.objective)}")

Status do problema: Optimal
u_('J',_'G3')=1.0
u_('J',_'G5')=1.0
u_('L',_'G4')=1.0
u_('Ma',_'G1')=1.0
u_('Ma',_'G2')=1.0
u_('Ma',_'G6')=1.0
u_('Mi',_'G0')=1.0
u_('V',_'G7')=1.0
x_('J',_'E0',_'D18')=1.0
x_('J',_'E10',_'D13')=1.0
x_('J',_'E18',_'D21')=1.0
x_('J',_'E19',_'D23')=1.0
x_('J',_'E2',_'D20')=1.0
x_('J',_'E24',_'D0')=1.0
x_('J',_'E25',_'D5')=1.0
x_('J',_'E26',_'D28')=1.0
x_('J',_'E27',_'D8')=1.0
x_('J',_'E28',_'D17')=1.0
x_('J',_'E29',_'D12')=1.0
x_('J',_'E3',_'D4')=1.0
x_('J',_'E30',_'D26')=1.0
x_('J',_'E31',_'D10')=1.0
x_('J',_'E33',_'D1')=1.0
x_('J',_'E34',_'D9')=1.0
x_('J',_'E35',_'D31')=1.0
x_('J',_'E36',_'D6')=1.0
x_('J',_'E37',_'D3')=1.0
x_('J',_'E38',_'D32')=1.0
x_('J',_'E39',_'D34')=1.0
x_('J',_'E40',_'D7')=1.0
x_('J',_'E41',_'D16')=1.0
x_('J',_'E42',_'D11')=1.0
x_('J',_'E43',_'D30')=1.0
x_('J',_'E44',_'D14')=1.0
x_('J',_'E45',_'D33')=1.0
x_('J',_'E5',_'D27')=1.0
x_('J',_'E55',_'D19')=1.0
x_('J',_'E56',_'D25')=1.0
x_('J',_'E57',_'D29')=1.0
x_('J',_'E59',_'D22')=1.0
x_('J

In [5]:
#Print results organized per day
for d in days:
    print(f"-------------------Day: {d}----------------------")
    for k in desks:
        for e in employee:
            if e in employee_per_desk[k]:
                if x[d, e, k].value() == 1:
                    print(f"Desk: {k}, employee:{e}")
    for g in groups:
        if u[d, g].value() == 1:
            group_employees = employee_per_group[g]
            print(f"Meeting of the day: Group {g} - {group_employees}")

-------------------Day: L----------------------
Desk: D0, employee:E0
Desk: D1, employee:E45
Desk: D2, employee:E50
Desk: D3, employee:E1
Desk: D4, employee:E52
Desk: D5, employee:E42
Desk: D6, employee:E39
Desk: D7, employee:E40
Desk: D8, employee:E56
Desk: D9, employee:E37
Desk: D10, employee:E58
Desk: D11, employee:E34
Desk: D12, employee:E27
Desk: D13, employee:E30
Desk: D14, employee:E33
Desk: D15, employee:E38
Desk: D16, employee:E25
Desk: D17, employee:E35
Desk: D18, employee:E46
Desk: D19, employee:E10
Desk: D20, employee:E51
Desk: D21, employee:E8
Desk: D22, employee:E20
Desk: D23, employee:E21
Desk: D24, employee:E14
Desk: D25, employee:E2
Desk: D26, employee:E36
Desk: D27, employee:E11
Desk: D28, employee:E32
Desk: D29, employee:E7
Desk: D30, employee:E16
Desk: D31, employee:E23
Desk: D32, employee:E22
Desk: D33, employee:E54
Desk: D34, employee:E55
Meeting of the day: Group G4 - ['E32', 'E33', 'E34', 'E35', 'E36', 'E37', 'E38']
-------------------Day: Ma--------------------

In [6]:
# --- FIM DA MEDIÇÃO ---
end_time = time.time()

# Calcula o tempo total decorrido
time_taken = end_time - start_time

# Imprime o tempo de execução formatado
print(f"\n--- Tempo de Execução do Código ---")
print(f"O processo completo levou {time_taken:.4f} segundos para ser executado.")
print(f"-----------------------------------")


--- Tempo de Execução do Código ---
O processo completo levou 100.4780 segundos para ser executado.
-----------------------------------
